<a href="https://colab.research.google.com/github/catyung/AI-Marketer/blob/main/integration/Slack_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLACK INTEGRATE

In [ ]:
!pip install slack_sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os
os.environ["SLACK_USER_TOKEN"] = userdata.get('SLACK_USER_TOKEN')
os.environ["SLACK_BOT_TOKEN"] = userdata.get('SLACK_BOT_TOKEN')

In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

async def reply_to_user_in_dm(user_id, message):
    """
    Replies to a user in a private direct message.

    Args:
        user_id (str): The ID of the user to send the message to.
        message (str): The message to send.
    """
    # Get the Slack bot token from an environment variable
    slack_bot_token = os.environ.get("SLACK_BOT_TOKEN")

    # Create a Slack client
    client = WebClient(token=slack_bot_token)

    try:
        # Open a direct message conversation with the user
        response = client.conversations_open(users=user_id)
        channel_id = response["channel"]["id"]

        # Send the message to the user in the direct message conversation
        response = client.chat_postMessage(
            channel=channel_id,
            text=message
        )
        print(f"Message sent: {response['message']}")
    except SlackApiError as e:
        print(f"Error sending message: {e}")

In [ ]:
reply_to_user_in_dm("U06QTRTCNJE", "Hello, user111!")

Message sent: {'user': 'U06QUSYK0RG', 'type': 'message', 'ts': '1713502656.745679', 'bot_id': 'B06QN858E4W', 'app_id': 'A06QN7QAEFQ', 'text': 'Hello, user111!', 'team': 'T06QQU0SKEZ', 'bot_profile': {'id': 'B06QN858E4W', 'app_id': 'A06QN7QAEFQ', 'name': 'Slack Exporter', 'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png', 'image_48': 'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png', 'image_72': 'https://a.slack-edge.com/80588/img/plugins/app/service_72.png'}, 'deleted': False, 'updated': 1711099219, 'team_id': 'T06QQU0SKEZ'}, 'blocks': [{'type': 'rich_text', 'block_id': 'W8A', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hello, user111!'}]}]}]}


# FLASK SERVER

In [ ]:
!pip install pyngrok
!ngrok authtoken 2ZppXjPSIKrVg9f1P1Melzo1Cvm_6hNALRoMdjWMzcW26DztP
! curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!pip install markupsafe==2.0.1

In [ ]:
from flask import Flask, request, redirect
import threading

app = Flask(__name__)
from pyngrok import ngrok
port = "5000"
public_url = ngrok.connect(port).public_url
print(f"* ngrok tunnel {public_url} -> http://127.0.0.1:{port}")



# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
@app.route('/', methods=['POST'])
def callback():
    print(request.json)
    try:
        return request.json['challenge']
    except:
        pass

    try:
        if 'client_msg_id' in request.json['event'].keys():
            reply_to_user_in_dm(request.json['event']['user'], 'Your Message:' + str(request.json['event']['text']))
            return 'Finished'
    except:
        pass
    return 'Finished'

* ngrok tunnel https://c5c5-34-138-21-126.ngrok-free.app -> http://127.0.0.1:5000


In [ ]:
threading.Thread(app.run()).start()

# FASTAPI SERVER

notebook cannot run async, please refer to server.py

In [ ]:
!pip install fastapi
!pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, Request
import threading
from pyngrok import ngrok

app = FastAPI()

port = "5000"
#public_url = ngrok.connect(port).public_url
public_url = 'https://997f-34-138-21-126.ngrok-free.app'
print(f"* ngrok tunnel {public_url} -> http://127.0.0.1:{port}")

# Update any base URLs to use the public ngrok URL
app.state.BASE_URL = public_url

@app.post("/")
async def callback(request: Request):
    try:
        data = await request.json()
        print(data)
        return data['challenge']
    except:
        pass

    try:
        if 'client_msg_id' in data['event'].keys():
            await reply_to_user_in_dm(data['event']['user'], 'Your Message:' + str(data['event']['text']))
            return 'Finished'
    except:
        pass

    return 'Finished'

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=int(port))

* ngrok tunnel https://997f-34-138-21-126.ngrok-free.app -> http://127.0.0.1:5000


RuntimeError: asyncio.run() cannot be called from a running event loop